<a href="https://colab.research.google.com/github/siddharath28/Excel-Sales-Analytics/blob/main/process2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os

def process_gps_data(input_file, output_dir):

    df = pd.read_parquet(input_file)


    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')


    df = df.sort_values(by=['unit', 'timestamp'])


    for unit, unit_data in df.groupby('unit'):
        current_trip_start = unit_data.iloc[0]['timestamp']
        current_trip_number = 1


        for i in range(1, len(unit_data)):
            time_diff = unit_data.iloc[i]['timestamp'] - unit_data.iloc[i - 1]['timestamp']


            if time_diff > timedelta(hours=7):

                trip_file_name = f"{unit}_trip_{current_trip_number}.csv"
                trip_file_path = os.path.join(output_dir, trip_file_name)


                unit_data.iloc[i - 1].to_frame().T.to_csv(trip_file_path, index=False)


                current_trip_start = unit_data.iloc[i]['timestamp']
                current_trip_number += 1


        trip_file_name = f"{unit}_trip_{current_trip_number}.csv"
        trip_file_path = os.path.join(output_dir, trip_file_name)
        unit_data.iloc[-1].to_frame().T.to_csv(trip_file_path, index=False)

if __name__ == "__main__":
    # Example usage:
    input_file = "raw_data.parquet"
    output_directory = "output_trips"


    os.makedirs(output_directory, exist_ok=True)

    process_gps_data(input_file, output_directory)


In [4]:
import os
import json
import csv
import sys
import argparse

def process_json_files(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # List all JSON files in the input directory
    json_files = [f for f in os.listdir(input_dir) if f.endswith('.json')]

    # Create the CSV file for consolidated data
    csv_file_path = os.path.join(output_dir, 'transformed_data.csv')
    with open(csv_file_path, 'w', newline='') as csv_file:
        # Define CSV header
        fieldnames = ['unit', 'trip_id', 'toll_loc_id_start', 'toll_loc_id_end',
                      'toll_loc_name_start', 'toll_loc_name_end', 'toll_system_type',
                      'entry_time', 'exit_time', 'tag_cost', 'cash_cost', 'license_plate_cost']

        # Create a CSV writer
        csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Write header to the CSV file
        csv_writer.writeheader()

        # Process each JSON file
        for json_file in json_files:
            # Generate trip_id from file name
            trip_id = os.path.splitext(json_file)[0]

            # Read JSON data from the file
            json_file_path = os.path.join(input_dir, json_file)
            with open(json_file_path, 'r') as json_file:
                data = json.load(json_file)

            # Extract relevant data from JSON
            unit = data.get('unit', '')
            tolls = data.get('tolls', [])

            # If there are no tolls in a route, ignore the file
            if not tolls:
                continue

            # Process each toll entry
            for toll in tolls:
                entry_time = toll.get('entry_time', '')
                exit_time = toll.get('exit_time', '')
                tag_cost = toll.get('tag_cost', '')
                cash_cost = toll.get('cash_cost', '')
                license_plate_cost = toll.get('license_plate_cost', '')
                toll_loc_start = toll.get('toll_loc_start', {})
                toll_loc_end = toll.get('toll_loc_end', {})

                # Write data to CSV
                csv_writer.writerow({
                    'unit': unit,
                    'trip_id': trip_id,
                    'toll_loc_id_start': toll_loc_start.get('toll_loc_id', ''),
                    'toll_loc_id_end': toll_loc_end.get('toll_loc_id', ''),
                    'toll_loc_name_start': toll_loc_start.get('toll_loc_name', ''),
                    'toll_loc_name_end': toll_loc_end.get('toll_loc_name', ''),
                    'toll_system_type': toll.get('toll_system_type', ''),
                    'entry_time': entry_time,
                    'exit_time': exit_time,
                    'tag_cost': tag_cost,
                    'cash_cost': cash_cost,
                    'license_plate_cost': license_plate_cost
                })

# Run the script with example arguments
process_json_files("/content/", "/content/")
